In [47]:
# Load in data and packages

import pandas as pd
from business_rule_engine import RuleParser
import numpy as np
# import numpy as np
Batter_Data=pd.read_csv("E:\\batter_view_sample.csv") # Result of Athena Query

In [33]:
# Function use is to take values that qualify for a note and append to an already existing dataframe
## This dataframe will hold all the values that qualify for a note by the completion of the program
def AppendToRules(event_id,pbr_id,NoteID,Value1,Value2,Value3,Rank1,Rank2,Rank3):
    global Notes
    Row=[event_id,pbr_id,NoteID,Value1,Value2,Value3,Rank1,Rank2,Rank3]
    Notes.loc[len(Notes.index)] = Row
    return Notes

In [2]:
# Position player rules

## This is where we define what the rules are for qualifying for a note
### The engine definition of rules requires 3 things: 1) The rule name 2) The condition and 3) what to do when the condition is met
### Here, we check the metrics from the athena query to see if they satisfy the condition for a note, if they do, we append them to the notes dataframe by using the function above
rules = """
rule "national bat ev"
when
    AND(avg_bat_speed_overall_percentile >= 0.95,
    avg_exitspeed_overall_percentile >= 0.95)
then
    AppendToRules(event_id,player_id,1,avg_bat_speed,avg_exitspeed,'',avg_bat_speed_overall_percentile,avg_exitspeed_overall_percentile,'')
end

rule "spinrate_event_rank"
when
    AND(avg_exitspeed_overall_percentile >= 0.95,
    avg_rotational_acceleration_overall_percentile >= 0.95)
then
    AppendToRules(event_id,player_id,2,avg_exitspeed,avg_rotational_acceleration,'',avg_exitspeed_overall_percentile,avg_rotational_acceleration_overall_percentile,'')
end

rule "event_hitting_triplet"
when
    AND(event_avg_distance_rank <= 3,
    event_avg_hard_ball_percentage_rank <= 3,
    event_sweetspot_percentage_rank <= 3)
then
    AppendToRules(event_id,player_id,3,avg_distance,hard_ball_percentage,sweetspot_percentage,event_avg_distance_rank,event_avg_hard_ball_percentage_rank,event_sweetspot_percentage_rank)
end

rule "ev_sweetspot"
when
    AND(event_avg_exitspeed_rank <= 3,
    event_sweetspot_percentage_rank <= 3)
then
    AppendToRules(event_id,player_id,4,avg_exitspeed,sweetspot_percentage,"",event_avg_exitspeed_rank,event_sweetspot_percentage_rank,"")
end

rule "ev_100_percentile"
when
    max_exitspeed >= 100
then
    AppendToRules(event_id,player_id,5,max_exitspeed,"","",max_exitspeed_overall_percentile,"","")
end

rule "ev_105_percentile"
when
    max_exitspeed >= 105
then
    AppendToRules(event_id,player_id,6,max_exitspeed,"","",max_exitspeed_overall_percentile,"","")
end

rule "sixty_batspeed_handspeed_hardhit" #Note, we are not giving any info about hard hit, only that it's in the top 50%, so we can get away with not reporting it in the notes dataframe
when
    AND(avg_bat_speed_overall_percentile >= 0.75,
    avg_peak_hand_speed_overall_percentile >= 0.75,
    sixty_overall_percentile >= 0.75,
    hard_ball_percentage_overall_percentile >= 0.5)
then
    AppendToRules(event_id,player_id,7,avg_bat_speed,avg_peak_hand_speed,sixty,avg_bat_speed_overall_percentile,avg_peak_hand_speed_overall_percentile,sixty_overall_percentile)
end

rule "triple blast event leaders"
when
    AND(event_avg_rotational_acceleration_rank = 1,
    event_avg_bat_speed_rank = 1,
    event_avg_on_plane_efficiency_rank)
then
    AppendToRules(event_id,player_id,8,avg_rotational_acceleration,avg_bat_speed,avg_on_plane_efficiency,1,1,1)
end

rule "ev_sixty_top3"
when
    AND(event_avg_exitspeed_rank <= 3,
    event_sixty_rank <= 3)
then
    AppendToRules(event_id,player_id,9,avg_exitspeed,sixty,"",event_avg_exitspeed_rank,event_sixty_rank,"")
end

rule "ra_position_velo_top3" # we're getting null values because of the max value. How to fix this???
when
    AND(event_avg_rotational_acceleration_rank <= 3,
    OR(event_catcher_velo_rank <= 3,
        event_infield_velo_rank <= 3,
        event_outfield_velo_rank <= 3))
then
    AppendToRules(event_id,player_id,10,sixty,max(catcher_velo,infield_velo,outfield_velo),"",event_avg_exitspeed_rank,min(event_catcher_velo_rank,event_infield_velo_rank,event_outfield_velo_rank),"")
end

rule "ra_batspeed_top3"
when
    AND(event_avg_rotational_acceleration_rank <= 3,
    event_avg_bat_speed_rank <= 3)
then
    AppendToRules(event_id,player_id,11,avg_rotational_acceleration,avg_bat_speed,"",event_avg_exitspeed_rank,event_avg_bat_speed_rank,"")
end

rule "batspeed_rotationalacceleration_differences"
when
    AND(avg_bat_speed_overall_percentile >= 0.9,
    avg_rotational_acceleration_overall_percentile <= 0.4)
then
    AppendToRules(event_id,player_id,12,avg_bat_speed,avg_rotational_acceleration,"",avg_bat_speed_overall_percentile,avg_rotational_acceleration_overall_percentile,"")
end

rule "top_sixty_batspeed_peakev"
when
    AND(event_sixty_rank = 1,
    event_avg_bat_speed_rank = 1,
    event_max_exitspeed_rank = 1)
then
    AppendToRules(event_id,player_id,13,sixty,avg_bat_speed,max_exitspeed,event_sixty_rank,event_avg_bat_speed_rank,event_max_exitspeed_rank)
end

rule "event_catcher_ev_poptime"
when
    AND(event_avg_bat_speed_rank = 1,
    event_max_exitspeed_rank = 1)
then
    AppendToRules(event_id,player_id,14,sixty,avg_bat_speed,max_exitspeed,event_sixty_rank,event_avg_bat_speed_rank,event_max_exitspeed_rank)
end

rule "ra_raw"
when
    avg_rotational_acceleration >= 16.8
then
    AppendToRules(event_id,player_id,15,avg_rotational_acceleration,"","",event_avg_rotational_acceleration_rank,"","")
end


"""

In [4]:
# Now that we've defined the rules, we actually run the rules engine

# Create an empty dataframe to hold the notes once they satisfy the condition to become a note
Notes=pd.DataFrame(columns=['event_id','pbr_id','NoteID','Value1','Value2','Value3','Rank1','Rank2','Rank3'])

# Prepare the rules engine
parser=RuleParser() #Create a parser to work with the existing rules
parser.register_function(AppendToRules) #Register the custom function we defined so the rules engine knows what it is whenever we run the code
parser.parsestr(rules) #Finally, add the string of rules to the parser

# Run the rules engine by looping through each row of the dataframe
for index,row in Batter_Data.iterrows():
    params=row.to_dict() #Convert the data into a dictionary to be used by the engine
    parser.execute(params) #Finally, execute the rules engine to run the program

In [2]:
Data=pd.read_csv("E:\\notes_pitcher_rankings.csv") #Read in data for the pitcher's notes

In [30]:
# We want to transform the data so that each player id and event_id has one and only one record.
## Whenever we read in from the view, each pitcher has duplicated rows, one for each pitch type, we want to consilidate that to one row

# Group by common pitch type variations and filter into individual dataframes for merging
Fastball=Data[(Data['taggedpitchtype'] == 'Fastball') | (Data['taggedpitchtype'] == 'Cutter') | (Data['taggedpitchtype'] == 'Sinker')]
Curveball=Data[Data['taggedpitchtype'] == 'Curveball']
Slider=Data[Data['taggedpitchtype'] == 'Slider']
Changeup=Data[(Data['taggedpitchtype'] == 'ChangeUp') | (Data['taggedpitchtype'] == 'Splitter') | (Data['taggedpitchtype'] == 'Knuckleball')]

In [3]:
# Rename columns appropriately for each dataframe
Fastball=Fastball.drop('taggedpitchtype',1) #Drop the taggedpitchtype (we already know it's a fastball)
Fastball=Fastball.add_prefix('fastball_') #Add a prefix so we know that the metrics are for a fastball
Fastball = Fastball.rename(columns={'fastball_event_id': 'event_id', 'fastball_player_id': 'player_id'}) #Keep event_id and player_id as before, we need these to be the same to merge on

Curveball=Curveball.drop('taggedpitchtype',1)
Curveball=Curveball.add_prefix('curveball_')
Curveball = Curveball.rename(columns={'curveball_event_id': 'event_id', 'curveball_player_id': 'player_id'})

Slider=Slider.drop('taggedpitchtype',1)
Slider=Slider.add_prefix('slider_')
Slider = Slider.rename(columns={'slider_event_id': 'event_id', 'slider_player_id': 'player_id'})

Changeup=Changeup.drop('taggedpitchtype',1)
Changeup=Changeup.add_prefix('changeup_')
Changeup = Changeup.rename(columns={'changeup_event_id': 'event_id', 'changeup_player_id': 'player_id'})

In [32]:
#Finally, merge all the dataframes together
Together=pd.merge(Fastball,Curveball,how='outer',on=['event_id','player_id'])
Together=pd.merge(Together,Slider,how='outer',on=['event_id','player_id'])
Together=pd.merge(Together,Changeup,how='outer',on=['event_id','player_id'])

In [77]:
# Position player rules --- the same as with the batter rules


# When trying to use np.nanmax() or np.nanmin() I get a NotImplementedError. Can we get around this???

### Rules that we don't have built yet
#   Also think about passing pitch type to the notes
#   We have many notes on the google sheet that don't have a filter or any indication on what we're looking for, get clarification
#   MLB average spin broken down by pitch type
#   Missing all DL new metrics (hop, rise, etc.)
#   Do not have D1 averages

rules = """
rule "event velo and spin"
when
    AND(fastball_event_max_velo_rank <= 3,
    OR(curveball_event_peak_spinrate_rank <= 3, slider_event_peak_spinrate_rank <= 3))
then
    AppendToRules(event_id,player_id,100,fastball_max_relspeed,max(curveball_peak_spinrate,slider_peak_spinrate),"",fastball_event_max_velo_rank,min(curveball_event_peak_spinrate_rank,slider_event_peak_spinrate_rank),"")
end

rule "event spins"
when
    AND(fastball_event_peak_spinrate_rank <= 3,
    OR(curveball_event_peak_spinrate_rank <= 3, slider_event_peak_spinrate_rank <= 3))
then
    AppendToRules(event_id,player_id,101,fastball_peak_spinrate,max(curveball_peak_spinrate,slider_peak_spinrate),"",fastball_event_peak_spinrate_rank,min(curveball_event_peak_spinrate_rank,slider_event_peak_spinrate_rank),"")
end

rule "national velo"
when
    fastball_max_relspeed_overall_percentile >= 0.95
then
    AppendToRules(event_id,player_id,102,fastball_max_relspeed,"","",fastball_max_relspeed_overall_percentile,"","")
end

rule "hop over 100"
when
    fastball_hop_plus >= 100
then
    AppendToRules(event_id,player_id,103,fastball_hop_plus,"","","","","")
end

rule "rise over 100"
when
    fastball_rise_plus >= 100
then
    AppendToRules(event_id,player_id,104,fastball_rise_plus,"","","","","")
end

rule "top velo, low spin"
when
    AND(fastball_event_max_velo_rank = 1,
    fastball_avg_spinrate_overall_percentile <= 0.25)
then
    AppendToRules(event_id,player_id,105,fastball_max_relspeed,fastball_avg_spinrate,"",1,fastball_avg_spinrate_overall_percentile,"")
end

rule "top velo, top spin"
when
    AND(fastball_event_max_velo_rank = 1,
    fastball_event_avg_spin_rank = 1)
then
    AppendToRules(event_id,player_id,106,fastball_max_relspeed,fastball_avg_spinrate,"",1,1,"")
end

rule "above average hop, above average hammer"
when
    AND(fastball_hop_plus >= 100,
    OR(changeup_hammer_plus >= 100, curveball_hammer_plus >= 100, slider_hammer_plus >= 100))
then
    AppendToRules(event_id,player_id,107,fastball_hop_plus,max(changeup_hammer_plus,curveball_hammer_plus,slider_hammer_plus),"","","","")
end

rule "above average rise, above average rise"
when
    AND(OR(fastball_rise_plus >= 100, changeup_rise_plus >= 100, slider_rise_plus >= 100, curveball_rise_plus >= 100),
    OR(changeup_hammer_plus >= 100, curveball_hammer_plus >= 100, slider_hammer_plus >= 100))
then
    AppendToRules(event_id,player_id,108,max(fastball_rise_plus >= 100, changeup_rise_plus >= 100, slider_rise_plus >= 100, curveball_rise_plus >= 100),max(changeup_hammer_plus,curveball_hammer_plus,slider_hammer_plus),"","","","")
end

rule "max fastball greater DL"
when
    fastball_max_relspeed >= 98.7
then
    AppendToRules(event_id,player_id,109,fastball_max_relspeed,"","",fastball_event_max_velo_rank,"","")
end

rule "fastball spin greater DL"
when
    fastball_avg_spinrate >= 2215
then
    AppendToRules(event_id,player_id,110,fastball_avg_spinrate,"","",fastball_event_avg_spin_rank,"","")
end

rule "max curveball greater DL"
when
    curveball_max_relspeed >= 69.3
then
    AppendToRules(event_id,player_id,111,curveball_max_relspeed,"","",curveball_event_max_velo_rank,"","")
end
"""

In [2]:
# Same as for the batter notes, will want to name as something different other than "Notes" as to not overwrite the notes from the batter
Notes=pd.DataFrame(columns=['event_id','pbr_id','NoteID','Value1','Value2','Value3','Rank1','Rank2','Rank3'])

parser=RuleParser()
parser.register_function(AppendToRules)
parser.parsestr(rules)

for index,row in Together.iterrows():
    params=row.to_dict()
    parser.execute(params)

In [46]:
# Casting various values to int instead of doubles
convert={
    'event_id': int,
    'pbr_id': int,
    'NoteID': int
}
Notes.astype(convert).sort_values(by=['NoteID'],ascending=False)

,event_id,pbr_id,NoteID,Value1,Value2,Value3,Rank1,Rank2,Rank3
0,10089,243921,100,79.3101,2471.1885,,1.0,3.0,
1518,14116,217056,100,91.8903,2652.1361,,3.0,1.0,
1512,14171,348046,100,77.5983,1966.3579,,1.0,2.0,
1513,14172,245969,100,78.3796,2159.7348,,3.0,1.0,
1514,14235,279434,100,80.6596,2052.1180,,3.0,3.0,
...,...,...,...,...,...,...,...,...,...
752,12223,312954,100,85.0916,2412.4724,,2.0,2.0,
751,12653,275207,100,84.1387,NaN,,2.0,NaN,
750,12649,258984,100,75.1905,1926.6705,,2.0,2.0,
749,12649,277044,100,83.2313,2190.8296,,1.0,1.0,
